In [18]:
# !pip install transformers
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip3 install jupyterlab
# !pip install onnx
# !pip install onnxsim

https://huggingface.co/docs/transformers/model_doc/gpt2

In [16]:
from transformers import GPT2Model, GPT2Tokenizer
import torch
from pathlib import Path

model_name = "distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2Model.from_pretrained(model_name)

# ONNXファイルの保存先
onnx_output_path = Path("distilgpt2_20230307.onnx")

# ダミー入力の準備
input_ids = torch.tensor([tokenizer.encode("Hello, my dog is cute")])

# ONNXエクスポート実行
torch.onnx.export(model, 
                  input_ids, 
                  onnx_output_path, 
                  export_params=True, 
                  opset_version=11, 
                  do_constant_folding=True, 
                  input_names=['input_ids'], 
                  output_names=['outputs'],
                  dynamic_axes={'input_ids': {0: 'batch_size'}, 'outputs': {0: 'batch_size'}})


In [17]:
!onnxsim distilgpt2_20230307.onnx distilgpt2_20230307_simplified.onnx

Installing onnxruntime by `C:\Users\thyt\envs\huggingface\Scripts\python.exe -m
pip install onnxruntime`, please wait for a moment..
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)
  Using cached pyreadline3-3.4.1-py3-none-any.whl (95 kB)
Your model contains "Tile" ops or/and "ConstantOfShape" ops. Folding these ops 
can make the simplified model much larger. If it is not expected, please 
specify "--no-large-tensor" (which will lose some optimization chances)
Simplifying...
Finish! Here is the difference:
┌─────────────────┬────────────────┬──────────────────┐
│                 │ Original Model │ Simplified Model │
├─────────────────┼────────────────┼──────────────────┤
│ Add             │ 57             │ 51               │
│ Cast            │ 19             │ 0                │
│ Concat          │ 74             │ 0                │
│ Constant

You should consider upgrading via the 'C:\Users\thyt\envs\huggingface\Scripts\python.exe -m pip install --upgrade pip' command.


### ssc4onnx

In [19]:
!pip install -U onnx rich onnxruntime && pip install -U ssc4onnx && python -m pip install onnx_graphsurgeon --index-url https://pypi.ngc.nvidia.com

Looking in indexes: https://pypi.ngc.nvidia.com


You should consider upgrading via the 'C:\Users\thyt\envs\huggingface\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\thyt\envs\huggingface\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\Users\thyt\envs\huggingface\Scripts\python.exe -m pip install --upgrade pip' command.


In [23]:
!ssc4onnx -if distilgpt2_20230307.onnx

┌────────────────────────┬────────────┬────────────┐
│ OP Type                │ OPs        │ Sizes      │
├────────────────────────┼────────────┼────────────┤
│ Add                    │ 57         │ 39.0KiB    │
│ Cast                   │ 19         │ 532.0B     │
│ Concat                 │ 74         │ 1.7KiB     │
│ Constant               │ 420        │ 1.0MiB     │
│ ConstantOfShape        │ 6          │ 24.0B      │
│ Div                    │ 19         │ 0.0B       │
│ Gather                 │ 99         │ 150.2MiB   │
│ Gemm                   │ 24         │ 162.2MiB   │
│ MatMul                 │ 12         │ 0.0B       │
│ Mul                    │ 37         │ 39.0KiB    │
│ Pow                    │ 25         │ 0.0B       │
│ Range                  │ 1          │ 0.0B       │
│ ReduceMean             │ 26         │ 1.6KiB     │
│ Reshape                │ 74         │ 0.0B       │
│ Shape                  │ 140        │ 0.0B       │
│ Slice                  │ 55         │ 0.0B  

In [22]:
!ssc4onnx -if distilgpt2_20230307_simplified.onnx

┌────────────────────────┬────────────┬────────────┐
│ OP Type                │ OPs        │ Sizes      │
├────────────────────────┼────────────┼────────────┤
│ Add                    │ 51         │ 57.1KiB    │
│ Div                    │ 19         │ 24.0B      │
│ Gather                 │ 1          │ 147.2MiB   │
│ Gemm                   │ 24         │ 162.2MiB   │
│ MatMul                 │ 12         │ 0.0B       │
│ Mul                    │ 37         │ 39.1KiB    │
│ Pow                    │ 19         │ 76.0B      │
│ ReduceMean             │ 26         │ 1.6KiB     │
│ Reshape                │ 66         │ 1.5KiB     │
│ Softmax                │ 6          │ 168.0B     │
│ Split                  │ 6          │ 648.0B     │
│ Sqrt                   │ 13         │ 0.0B       │
│ Sub                    │ 13         │ 0.0B       │
│ Tanh                   │ 6          │ 0.0B       │
│ Transpose              │ 30         │ 2.6KiB     │
│ Where                  │ 6          │ 240.0B